# Project Sidewalk - get crossing features

In [ ]:
import json
import requests
import functions as f

import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame

import matplotlib.pyplot as plt

import seaborn as sns
import folium

import config_azure as cf

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

### Settings

In [ ]:
CRS = 'epsg:28992'

## 1. Get project sidewalk data

### Get the correct url for a small test neighborhood or the whole city

In [ ]:
base_url = "https://sidewalk-amsterdam.cs.washington.edu/v2/access/attributesWithLabels?lat1={}&lng1={}&lat2={}&lng2={}" 
amsterdam_bbox_coords = (52.303, 4.8, 52.425, 5.05)
url = base_url.format(*amsterdam_bbox_coords)

### Try to read the data from local dump if available else go to the Project Sidewalk API

In [ ]:
local_dump = url.replace('/', '|')

try:
    project_sidewalk_labels = json.load(open(local_dump, 'r'))
except Exception as e:
    print("Couldn't load local dump")
    project_sidewalk_labels = requests.get(url.format(*amsterdam_bbox_coords)).json()
    json.dump(project_sidewalk_labels, open(local_dump, 'w'))

### Create sidewalk data geodataframe

In [ ]:
p_sw_gdf = gpd.GeoDataFrame.from_features(project_sidewalk_labels['features'], crs='epsg:4326')
p_sw_gdf = p_sw_gdf.to_crs(CRS)
p_sw_gdf

### Get mask of area

In [ ]:
# Import areas
df_areas = gpd.read_file(cf.output_pilot_area)
polygon = df_areas.to_crs(crs=CRS).unary_union

### Extract sidewalk data in polygon of area

In [ ]:
sw_data_in_area_mask = polygon.contains(p_sw_gdf['geometry'])
p_sw_area_gdf = p_sw_gdf.loc[sw_data_in_area_mask]

### Inspect data

In [ ]:
if p_sw_area_gdf.empty:
    print('No sidewalk data available in area')

p_sw_area_gdf

In [ ]:
# Create tooltip for feature representation on map
def gen_tooltip(fields, aliases):

    tooltip = folium.GeoJsonTooltip(
        fields=fields,
        aliases=aliases,
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
    return tooltip

# set True for satelite background, False for regular background
satelite = False

# Set folium map background
if satelite == True:
    network_color = 'white'
    tile = folium.TileLayer(
                tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                attr = 'Esri',
                name = 'Esri Satellite',
                overlay = False,
                control = True)
else:
    tile = 'openstreetmap'
    network_color = 'black'

# Create Folium map
map = folium.Map(
    location=[52.389164, 4.908453], tiles=tile,
    min_zoom=10, max_zoom=25, zoom_start=15,
    zoom_control=True, control_scale=True, control=False
    )

# Add traffic sign features
feature_names = p_sw_area_gdf.columns.tolist()
feature_names.remove('geometry')
tooltip = gen_tooltip(feature_names, feature_names)
geo_j = folium.GeoJson(p_sw_area_gdf, tooltip=tooltip, style_function=lambda x: {"color": "orange", "weight": 10}).add_to(map)

map

### Save project sidewalk data in selected area as csv

In [ ]:
# Save project sidewalk crossing features as csv
p_sw_area_gdf.to_csv(cf.out_folder + '/crossing_features/project_sidewalk/crossing_features_project_sidewalk.csv')

# Save network with project sidewalk crossing features as html map
if satelite == True:
    map.save(cf.out_folder + '/crossing_features/project_sidewalk/crossing_features_project_sidewalk_satelite.html')
else:
    map.save(cf.out_folder + '/crossing_features/project_sidewalk/crossing_features_project_sidewalk.html')